# SHORE - Python wrapper for [OCEAN](https://feff.phys.washington.edu/OCEAN/index.html) code   
<img src="data/figures/img1a.png" alt="drawing" width="1000"/>

----

Key features for OCEAN:
- Accurate prediciton of the K and some L edges for the crystaline materials.
- Ab initio framework(Almost parameters free)
- No need for supercell -> fast

Key features for SHORE:
- Flexible
- Object Oriented

----

Install:

In [ ]:
pip install shore-pkg-geonda -U

In [ ]:
import shore

### The structure of the code:
<img src="data/figures/img2.png" alt="drawing" width="1000"/>


---------
#### What is inside the input object?
<img src="data/figures/img3.png" alt="drawing" width="1000"/>

--------
#### Input components: Structure

In [ ]:
cu=shore.AtomicStructure('data/structures/cu.cif')

Go ahead and visualise the structure

In [ ]:
cu.view()

Now let's inspect availible atributes and methods for the structure object

In [ ]:
cu.info()

-------

#### Input components: Matter part 

Create matter object using Matter() class. Structure has to be given to one of the keywords argument and any parameters for calcualtion of the DFT,BSE parts.

In [ ]:
matter=shore.Matter(structure=cu, screen_nbands=10,)

To see all parameters (some are defaults) use matter.show() method.

In [ ]:
matter.show()

Again to inspect one this object can and cannot do check .info() method.

In [ ]:
matter.info()

----

#### Input components: Light part

<img src="data/figures/img4.png" alt="drawing" width="1000"/>

The "light" object contains list of photons with various polarization, direction and energy defiend from the absorption edge of interest. First create the object.

In [ ]:
light=shore.Light()

Now let's create a photon with polarization along x axis and energy which is matching the Cu K-edge (1s-2p) resonant excitions. 

In [ ]:
photon1=shore.Photon(polarization=[1,0,0],q=[0,0,1], interaction='dipole', energy=dict(element="Cu", edge='K'))

To check whats inside photon object use .info() method.

In [ ]:
photon1.info()

Now second photon will have a different polarision. The direction of the wavevector is not defined here since were working in the dipole approximation for now.

In [ ]:
photon2=shore.Photon(polarization=[0,1,0],
    energy=dict(element="Cu", edge='K'))

Populate the "light" object with photons.

In [ ]:
light.add(photon1)
light.add(photon2)

Finally, check what is the content of the "light" part. 

In [ ]:
light.show()

------
#### Input object

Here we want to combine all the previously defined parts and create the 'input' object, which we will letter pass to the clalculation workflow. 

In [ ]:
ocean=shore.Input(name='ocean', matter=matter, light=light,)

In [ ]:
ocean.info()

To inspect finale list of parameters one could use .content.show() method. 

In [ ]:
ocean.content.show()

Congratulaitons you have finished preparing input object for the OCEAN calculations. To procced we have to set up connection to the HPC cluster and learn about aspects of the workflow and learn how to submit and retirieve calculations. 

------
### * Server

Before proceeding, we need to create one additional object: the server (HPC cluster) on which all calculations will be executed.

In [ ]:
# remote_host = "ip address of the cluster"   
remote_host = "10.16.77.14"   
username = "a.burov"                     
server=shore.RemoteServerManager(remote_host,username)

In oreder to reuse it we can write the whole object using pickle. By default it will write it to the jar folder in the current directory.

In [ ]:
server.save('myserver.pkl')

In case you want to read sever infomation from file, here we go:

In [ ]:
# server=shore.RemoteServerManager(load='path/to/myserver.pkl')

The Server object includes the following methods and attributes:

In [ ]:
server.info()

--------
#### Workflow
We now ready to create a workflow and sumbit our first calculation. As an input to the workflow object we are passing a server, so that it will inherite all the infomation and now were to sumbit the jobs. This part is not required to run the calculations but it could be handy to visulise existent calculations.

In [ ]:
pipe=shore.Workflow(server=server)

-----

Now let's finally move to the calculation of XAS. To create an instance of xas from the input file and for the given (pipeline,server) use:

In [ ]:
calc=shore.Calculation(workflow=pipe,input=ocean)

Check what have created:

In [ ]:
calc.info()

The are two impotant method in the instance. Calculation().run() submits the job to the HPC cluster, and Calculation().sync() retrieves the results once the calculations are done. 

<img src="data/figures/img5.png" alt="drawing" width="1000"/>

To viuslise this process as a graph we can use 

In [ ]:
pipe.show()

here green color means that the part is down and blue backgorund represent the stages were are suppose to run on the cluster. For now we will not submit any calculations but you can do using the following command:

In [ ]:
calc.run()

In [ ]:
calc.get_status()

#### Mutiple calculation in one pipeline

That could be usefull if you want to check convergence or have mulitple compounds. First create an empty dictionary which we will populate.

In [ ]:
calculations={}

The key point in the lines of code below is the fork() method of the input object. We are taking orginal input object `ocean` and createing a copy of it with modified parameters (in our case just the diferent names)

In [ ]:
for item in ['name1', 'name2', 'name3']:
    tmp=ocean.fork(name=item)
    calculations[item]=shore.Calculation(workflow=pipe, input=tmp)

Check what have changed in pipeline graph:

In [ ]:
pipe.show()

-----

### Get results

Once the calculations are done we can retrieve the results from the cluster. To do that you would need to use `Calculation().sync()` for example:

`test=calc.sync()`

Syncing Files: 100%|██████████| 5/5 [00:01<00:00,  3.55file/s]

Since we didn't actually sumbit anything yet, we will populate the `results` object from the pre-saved pickle file.

In [ ]:
calc.res=shore.ResultsHandler(load='data/ref/Cu/Cu.pkl')

You can visualize and access the raw the data easily in any fanshion you see convient, however there is a pre-build method to visualize using plotly library.

In [ ]:
from plotly import graph_objects as go

Create an empty figure

In [ ]:
fig=go.Figure()

Pass the figure to the keyword argument of the plot() method as well as element name and over details if nessesarly.

In [ ]:
calc.res.plot(fig=fig, element='Cu', core_level='1s')
fig.update_xaxes(range=[-10,20])

Great job!